In [16]:
import pandas as pd
from neuralprophet import NeuralProphet
from IPython.display import clear_output

In [17]:
# Carga de datos
df = pd.read_csv('sell-in.txt', sep='\t')
df['fecha'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')

In [18]:
# Función para crear la serie de tiempo para un producto
def serie_producto(id_producto):
    df_prod = df[df['product_id'] == id_producto].loc[:, ['fecha', 'tn']]
    df_prod = df_prod[(df_prod['fecha'].dt.year == 2019)]
    if len(df_prod) == 0:
        return None

    # Encontrar la fecha inicial y final para el producto
    fecha_inicial = df_prod['fecha'].min()
    fecha_final = df_prod['fecha'].max()
    
    # Generar fechas hasta febrero de 2020
    fechas_productos = pd.date_range(start=fecha_inicial, end='2020-02-01', freq='MS')
    df_fechas_productos = pd.DataFrame({'fecha': fechas_productos})
    
    df_prod = df_prod.groupby('fecha').agg({'tn': 'sum'}).reset_index()
    df_ret = pd.merge(df_fechas_productos, df_prod, on='fecha', how='left')
    df_ret['tn'].interpolate(method='linear', inplace=True)
    df_ret.fillna(0, inplace=True)
    return df_ret

In [19]:
# DataFrame para almacenar predicciones
df_pred = pd.read_csv('productos_a_predecir.txt', sep='\t')

In [20]:
# resultados = []

# #max_epochs = 40

# for i in range(len(df_pred)):
#     id_producto = df_pred.iloc[i, 0]
#     df_prod = serie_producto(id_producto)
#     clear_output()
#     print(f"Progress: {i+1}/{len(df_pred)}", end="\r")
#     if df_prod is not None:
#         df_prod_neuralprophet = df_prod.rename(columns={'fecha': 'ds', 'tn': 'y'})
        
#         modelo = NeuralProphet()
#         metrics = modelo.fit(df_prod_neuralprophet, early_stopping=True, progress = None)
        
#         future = modelo.make_future_dataframe(df_prod_neuralprophet, periods=1, n_historic_predictions=True)
#         forecast = modelo.predict(future)
        
#         pred_final = abs(forecast['yhat1'].iloc[-1])  # Valor absoluto para evitar negativos
#         resultados.append([id_producto, pred_final])

In [21]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def process_row(i):
    id_producto = df_pred.iloc[i, 0]
    df_prod = serie_producto(id_producto)
    
    if df_prod is not None:
        df_prod_neuralprophet = df_prod.rename(columns={'fecha': 'ds', 'tn': 'y'})
        
        modelo = NeuralProphet()
        metrics = modelo.fit(df_prod_neuralprophet, early_stopping=True, progress=None, epochs = 100)
        
        future = modelo.make_future_dataframe(df_prod_neuralprophet, periods=1, n_historic_predictions=True)
        forecast = modelo.predict(future)
        
        pred_final =  max(forecast['yhat'].iloc[-1],0) # Valor absoluto para evitar negativos
        return id_producto, pred_final
    return id_producto, None

resultados = []

# Create a thread pool executor
with ThreadPoolExecutor() as executor:
    # Submit tasks to the executor
    future_to_index = {executor.submit(process_row, i): i for i in range(len(df_pred))}
    
    # Process the results as they complete
    for future in as_completed(future_to_index):
        i = future_to_index[future]
        try:
            result = future.result()
            if result[1] is not None:  # Check if result is not None
                resultados.append(result)
        except Exception as e:
            print(f"Row {i} generated an exception: {e}")
        
        # Update progress
        clear_output(wait=True)
        print(f"Progress: {i+1}/{len(df_pred)}", end="\r")
        
print("\nProcessing complete!")


Progress: 780/780
Processing complete!


In [24]:
# Crear DataFrame con los resultados
df_resultados = pd.DataFrame(resultados, columns=['product_id', 'tn'])

# Guardar predicciones
df_resultados.to_csv('pred_neuralprophet0_1.csv', index=False)